In [ ]:
import asyncio

from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder, SystemMessagePromptTemplate, AIMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, ToolMessage, RemoveMessage, BaseMessage
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnableLambda, RunnableSequence, RunnableGenerator, RunnableMap

from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

## Define a prompt -> LLM -> Output Parser

In [ ]:
prompt_template = PromptTemplate.from_template(
    template="""You are a {role} assistant.

Question: {question}
Answer:"""
)

prompt_value = prompt_template.invoke(
    {"role": "bad", "question": "What is the capital of France?"}
)

prompt_value

In [ ]:
chat_prompt_tpl = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(
        template="""You are a {role} assistant. You need to extract the first name and last name of the person from the question.
Format into a JSON object with the keys 'first_name' and 'last_name'."""
    ),
    HumanMessage(content="Hi my name is John Doe")
])

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Step 1: Generate the prompt
prompt_value = chat_prompt_tpl.invoke(
    {
        "role": "bad",
        "question": "What is the capital of France?",
        "messages": [AIMessage(content="I don't know")],
    }
)

# Step 2: Generate the LLM call
llm_call = await llm.ainvoke(prompt_value)
print(llm_call)

# Step 3: Parse the LLM call
parser = JsonOutputParser()
parsed_llm_call = parser.invoke(llm_call)
parsed_llm_call["first_name"], parsed_llm_call["last_name"]

## Runnable Syntax Suger

In [ ]:
chat_prompt_tpl = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(
            template="""You are a {role} assistant. You need to extract the first name and last name of the person from the question.
Format into a JSON object with the keys 'first_name' and 'last_name'."""
        ),
        HumanMessage(content="Hi my name is John Doe"),
    ]
)

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

parser = JsonOutputParser()

"""
chain = RunnableSequence([
    chat_prompt_tpl,
    llm,
    parser
])
"""

# operator override
chain = chat_prompt_tpl | llm | parser

"""
def pipe(left, right): ...

chain = pipe(pipe(chat_prompt_tpl, llm), parser)
"""

result = chain.invoke({"role": "bad", "question": "What is the capital of France?"})
result

## Batching

In [ ]:
results = chain.batch([
    {"role": "bad", "question": "What is the capital of France?"},
    {"role": "helpful", "question": "What is the capital of Germany?"},
    {"role": "good", "question": "What is the capital of Italy?"},
])

results

In [ ]:
from langchain_core.messages import AIMessageChunk


def truncate_text(text: AIMessage):
    return text.content[:100]

# RunnableGenerator
async def truncate_text_stream(iter):
    total_length = 0
    async for chunk in iter:
        if total_length > 200:
            break

        total_length += len(chunk.content)
        yield chunk.content

    yield "END OF TEXT"

# async for chunk in llm.astream("Hi, tell me a long story"):
#     print(chunk.content, end="")

def capitalize(text: AIMessage):
    return text.content.upper()

async def capitalize_stream(iter):
    async for chunk in iter:
        yield chunk.content.upper()

chain_1 = llm | RunnableGenerator(truncate_text_stream)
async for chunk in chain_1.astream("Hi, tell me a long story"):
    print(chunk, end="")

In [ ]:
from operator import itemgetter

with open("doc.txt", "r") as f:
    context = f.read()

prompt = PromptTemplate.from_template(
    template="""
    You are a helpful assistant.
    Context: {context}
Extract all the useful API names from the context. Format in a JSON key value pairs, e.g. {{ "Runnable": "A class that does xyz", "RunnableLambda": "A class that does xyz" }}
        """
)

chain_1 = prompt | llm | JsonOutputParser()
chain_2 = prompt | llm | StrOutputParser()

chain = RunnableMap(
    {
        "structured": chain_1 | itemgetter("Messages"),
        "unstructured": chain_2 | (lambda x: x[:100]),
    }
)

# {
#     "structured": "explaination of prompt template",
#     "unstructured": "the first 100 characters of the unstructured output"
# }

# r = chain.invoke({"context": context})

# r['structured'] = r['structured']['PromptTemplate']
# r['unstructured'] = r['unstructured'][:100]

# chain.get_graph().print_ascii()

In [ ]:
!pip install grandalf

In [ ]:
from pydantic import BaseModel

class ApiItem(BaseModel):
    name: str
    description: str

class ApiList(BaseModel):
    items: list[ApiItem]
    name: str

prompt = PromptTemplate.from_template(
    template="""
    You are a helpful assistant.
    Context: {context}
    Extract all the useful API names from the context.
    """
)

llm = prompt | ChatOpenAI(model="gpt-4o-mini", temperature=0).with_structured_output(
    ApiList
)

async for chunk in llm.astream({ "context": context }):
    print(chunk)


"""
{
    "items": [
        {
            "name": "Runnable",
            "description": "A class that does xyz"
        }
    ]
    "name
}

items=[ApiItem(name='Installation')]
"""